<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/11_LLM_qlora_4bit_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMのファインチューニング: テキスト生成による分類

* `kunishou/databricks-dolly-15k-ja`というデータセットを使う
  * https://huggingface.co/datasets/kunishou/databricks-dolly-15k-ja
  * `category`が`classification`のrowだけを使う。
  * 分類問題とはいえ、テキストで答えるようになっている。

## インストール

In [ ]:
!pip install -U transformers trl peft datasets accelerate bitsandbytes

## 再現性の確保

In [ ]:
from transformers import set_seed

set_seed(0)

## 使用するLLMの指定

In [ ]:
model_id = "rinna/gemma-2-baku-2b-it"

## データセット

### データセットの取得

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
dataset = load_dataset("kunishou/databricks-dolly-15k-ja")
dataset = dataset.filter(lambda example: example["category"] == "classification")
dataset

### データセットをtraining/validation/test setsに分割する

In [ ]:
train_test_ds = dataset["train"].train_test_split(test_size=0.1, seed=1234)
valid_test_ds = train_test_ds["test"].train_test_split(test_size=0.5, seed=1234)

train_ds = train_test_ds["train"]
valid_ds = valid_test_ds["train"]
test_ds = valid_test_ds["test"]

print(len(train_ds), len(valid_ds), len(test_ds))

In [ ]:
train_ds[0]

## 学習のためのプロンプト

* 答えも含んだプロンプトを作る。
  * これを使って通常の言語モデルとしての学習をおこなう。

In [ ]:
def make_training_prompt(example):
  message = [
    {
      'role': 'user',
      'content': example['instruction']
    },
    {
      'role': 'model',
      'content': example['output']
    }
  ]
  return tokenizer.apply_chat_template(message, tokenize=False)

In [ ]:
make_training_prompt(train_ds[0])

### training setの前処理

In [ ]:
def add_text(example):
  example["text"] = make_training_prompt(example)
  return example

train_ds = train_ds.map(add_text)
train_ds = train_ds.remove_columns(["input", "category", "output", "index", "instruction"])

In [ ]:
train_ds[0]

### validation setの前処理

In [ ]:
valid_ds = valid_ds.map(add_text)
valid_ds = valid_ds.remove_columns(["input", "category", "output", "index", "instruction"])
valid_ds[0]

## 評価のためのプロンプト

* 評価の際はLLMに答えを作らせるので、答えを含まないプロンプトを作る。

In [ ]:
def make_eval_prompt(example):
  message = [
    {
      'role': 'user',
      'content': example['instruction']
    }
  ]
  return tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

In [ ]:
make_eval_prompt(test_ds[0])

### モデルに答えさせてみる

In [ ]:
import torch
from transformers import AutoModelForCausalLM

dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype=dtype,
).to("cuda:0")

In [ ]:
prompt = make_eval_prompt(test_ds[0])
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=300)
print(tokenizer.decode(outputs[0]))

In [ ]:
test_ds[0]

* ここでモデルの中身を見ておく。

In [ ]:
model

* モデルをいったん削除する。
  * GPUのメモリも解放する。

In [ ]:
del model
torch.cuda.empty_cache()

## 量子化＋LoRAによるfinetuning

### LoRAの設定

In [ ]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  inference_mode=False,
  r=8,
  lora_alpha=32,
  lora_dropout=0.1,
  target_modules=[
    "q_proj", "k_proj", "v_proj", "o_proj",
    #"gate_proj", "up_proj", "down_proj",
  ],
)

### 量子化の設定

In [ ]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16,
)

### モデルの読み込み

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype=dtype,
  low_cpu_mem_usage=True,
  quantization_config=bnb_config,
)

### 学習の設定

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./results",
  num_train_epochs=1,
  learning_rate=2e-4,
  max_steps=500,
  logging_steps=100,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  gradient_accumulation_steps=4,
  optim = 'adamw_torch',
  eval_strategy="steps",
  eval_steps=100,
)

### `SFTTrainer`の作成

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  train_dataset=train_ds,
  eval_dataset=valid_ds,
  dataset_text_field="text",
  args=training_args,
  max_seq_length=512, #512にしないとGoogle Colab無料版で走らない
)

In [ ]:
def print_trainable_parameters(model, verbose=False):
  trainable_params = 0
  all_param = 0
  for name, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
      if verbose: print(name)
  print(
      f"trainable params: {trainable_params} "
      f"|| all params: {all_param} "
      f"|| trainable%: {100 * trainable_params / all_param}"
  )

In [ ]:
print_trainable_parameters(trainer.model, verbose=True)

In [ ]:
trainer.train()

In [ ]:
lora_adaptor = "lora/" + model_id + "-QLoRA-4bit-double"
trainer.save_model(lora_adaptor)

In [ ]:
del model
torch.cuda.empty_cache()

In [ ]:
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="auto",
  quantization_config=bnb_config,
)

model = PeftModel.from_pretrained(model, lora_adaptor)

In [ ]:
prompt = make_eval_prompt(test_ds[0])
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=300)
print(tokenizer.decode(outputs[0]))

## 評価の仕方
* langchainのQAEvalChainを使う。
  * 説明は省略します。私自身は使ったことがないですので・・・。
  * 評価用のLLMとしては、OpenAIの`gpt-3.5-turbo-instruct`あたりに課金すると安いかも。